# Max-p Regionalization with Compactness Constraints

This notebook demonstrates how to use the max-p regionalization algorithm with compactness optimization.

## 1. Installation

First, install the required dependencies:

In [ ]:
# Uncomment and run this cell if you need to install dependencies
# !pip install numpy scipy pandas geopandas libpysal pysal matplotlib

## 2. Import Libraries

In [ ]:
import sys
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import libpysal
import matplotlib.pyplot as plt
from datetime import datetime
from scipy.spatial.distance import pdist, squareform
from copy import deepcopy

# Add src directory to path
sys.path.insert(0, '../src')
from compactness import Polygon, shpProc, Region, Partition

print("Libraries imported successfully!")

## 3. Load Sample Data

We'll use the Mexico dataset from libpysal as an example.

In [ ]:
# Download and load sample data from libpysal
mexico = libpysal.examples.load_example('mexico')
mexico_shp = mexico.get_path('mexicojoin.shp')

# Read the shapefile
gdf = gpd.read_file(mexico_shp)
print(f"Loaded {len(gdf)} polygons")
print(f"Columns: {list(gdf.columns)}")
gdf.head()

In [ ]:
# Visualize the data
fig, ax = plt.subplots(1, 1, figsize=(12, 8))
gdf.plot(ax=ax, edgecolor='black', facecolor='lightblue')
ax.set_title('Mexico States')
plt.show()

## 4. Set Parameters

In [ ]:
# Algorithm parameters
flag_rg = 1        # Enable compactness in region growth (0=off, 1=on)
flag_ea = 1        # Enable compactness in enclave assignment (0=off, 1=on)
flag_ls = 1        # Enable compactness in local search (0=off, 1=on)
randomGrow = 2     # Number of top candidates for random selection during growth
randomAssign = 2   # Number of top candidates for random selection during enclave assignment

# Threshold - minimum attribute value for each region
# Using PCGDP1940 (Per Capita GDP 1940) as threshold attribute
threshold = 5000   # Adjust based on your data

# Construction parameters
ITERCONSTRUCT = 100   # Number of construction iterations (reduce for faster demo)
ITERSA = 10           # Number of simulated annealing iterations

# Local search parameters
alpha = 0.998         # Cooling rate
tabuLength = 10       # Tabu list length
max_no_move = 50      # Maximum non-improving moves

# Attribute names (from Mexico dataset)
threshold_name = 'PCGDP1940'   # Threshold attribute
attrs_name_str = 'PCGDP1950'   # Attribute for heterogeneity calculation
attrs_name = [attrs_name_str]

print(f"Threshold: {threshold}")
print(f"Threshold attribute: {threshold_name}")
print(f"Heterogeneity attribute: {attrs_name_str}")

## 5. Prepare Data

In [ ]:
# Set environment variable for data path
os.environ['MAXP_DATA_PATH'] = os.path.dirname(mexico_shp) + '/'

# Create spatial weights matrix (Queen contiguity)
w = libpysal.weights.Queen.from_dataframe(gdf)
print(f"Spatial weights created: {w.n} observations")

# Calculate distance matrix for heterogeneity
attr = gdf[attrs_name].values
distance_matrix = squareform(pdist(attr, metric='cityblock'))
print(f"Distance matrix shape: {distance_matrix.shape}")

## 6. Initialize Shape Processing

In [ ]:
# Process shapes and calculate moment of inertia
print("Processing shapes...")
start_time = datetime.now()

Shp = shpProc(mexico_shp, attrs_name_str, threshold_name).polygons
n_polygons = len(Shp)

print(f"Processed {n_polygons} polygons")
print(f"Time elapsed: {datetime.now() - start_time}")

## 7. Run Region Growth Construction

In [ ]:
# Construction phase
print("Starting construction phase...")
maxp_time1 = datetime.now()

max_p = 0
partitions_list = []
arr = np.arange(0, n_polygons)
partition_id = 0

for iteration in range(ITERCONSTRUCT):
    if iteration % 20 == 0:
        print(f"Iteration {iteration}/{ITERCONSTRUCT}")
    
    C = 0  # Region counter
    enclaveList = []
    np.random.shuffle(arr)
    APartition = Partition(n_polygons)
    
    for index in range(n_polygons):
        P = arr[index]
        
        if APartition.label[P] != 0:
            continue
        
        # Grow region from seed P
        C += 1
        APartition.label[P] = C
        ARegion = Region(Shp, P, C, w, APartition.label)
        
        while len(ARegion.NeighborPolysID) > 0:
            if ARegion.isRegion(threshold):
                break
            combined_unit = ARegion.selectUnit_growRegion(Shp, APartition.label, w, flag_rg, randomGrow)
            APartition.label[combined_unit.id] = C
        
        if not ARegion.isRegion(threshold):
            C -= 1
            enclaveList = enclaveList + list(ARegion.data)
        else:
            ARegion.withinRegionHetero = ARegion.calculateWithinRegionHetero(distance_matrix)
            APartition.p += 1
            APartition.data.append(ARegion)
    
    # Update max_p and handle enclaves
    if APartition.p >= max_p:
        max_p = APartition.p
        
        # Assign enclaves
        enclave_index = 0
        while len(enclaveList) > 0:
            regionList = []
            AEnclave = enclaveList[enclave_index]
            ecNeighbors = w.neighbors[AEnclave]
            
            for ecn in ecNeighbors:
                if ecn not in enclaveList and APartition.label[ecn] not in regionList:
                    regionList.append(APartition.label[ecn])
            
            if len(regionList) == 0:
                enclave_index += 1
            else:
                if flag_ea == 1:
                    # Compactness-aware assignment
                    updatedRegionList = []
                    for regionID in regionList:
                        region = APartition.data[regionID - 1]
                        DiffShapeindex, rshape, shapeindex = region.enclaveAssign(Shp[AEnclave])
                        updatedRegionList.append((regionID, DiffShapeindex, rshape, shapeindex))
                    
                    updatedRegionList = sorted(updatedRegionList, key=lambda tup: tup[1])
                    top_n = min(len(updatedRegionList), randomAssign)
                    unit_index = np.random.randint(top_n) if top_n > 0 else 0
                    
                    min_region = updatedRegionList[unit_index][0]
                    min_rshape = updatedRegionList[unit_index][2]
                else:
                    # Random assignment
                    regionindex = np.random.randint(len(regionList))
                    min_region = regionList[regionindex]
                    region = APartition.data[min_region - 1]
                    _, min_rshape, _ = region.enclaveAssign(Shp[AEnclave])
                
                APartition.label[AEnclave] = min_region
                updateRegion = APartition.data[min_region - 1]
                updateRegion.data.add(AEnclave)
                updateRegion.area = min_rshape[1]
                updateRegion.centroidX = min_rshape[2]
                updateRegion.centroidY = min_rshape[3]
                updateRegion.inertia = min_rshape[0]
                updateRegion.spatialAttrTotal += Shp[AEnclave].threshold
                updateRegion.withinRegionHetero = updateRegion.calculateWithinRegionHetero(distance_matrix)
                del enclaveList[enclave_index]
                enclave_index = 0
        
        APartition.shapeindex = APartition.calculateShapeIndex()
        APartition.totalwithinRegionHetero = APartition.calculateWithinRegionHeteroTotal(distance_matrix)
        APartition.id = partition_id
        partitions_list.append(APartition)
        partition_id += 1

maxp_time2 = datetime.now()
print(f"\nConstruction completed in {maxp_time2 - maxp_time1}")
print(f"Maximum p found: {max_p}")
print(f"Number of partitions: {len(partitions_list)}")

## 8. Find Best Partition

In [ ]:
# Find the partition with best compactness
minPartitionShape = 1.0
bestPartition = None

for partition in partitions_list:
    if partition.p == max_p and partition.shapeindex < minPartitionShape:
        bestPartition = partition
        minPartitionShape = partition.shapeindex

print(f"Best partition shape index: {minPartitionShape:.4f}")
print(f"Number of regions: {bestPartition.p}")
print(f"Total within-region heterogeneity: {bestPartition.totalwithinRegionHetero:.2f}")

## 9. Visualize Results

In [ ]:
# Add region labels to geodataframe
gdf['region'] = bestPartition.label

# Plot the regionalization result
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Original data
gdf.plot(column=threshold_name, ax=axes[0], legend=True, 
         cmap='YlOrRd', edgecolor='black', linewidth=0.5)
axes[0].set_title(f'Original Data: {threshold_name}')
axes[0].axis('off')

# Regionalization result
gdf.plot(column='region', ax=axes[1], categorical=True, 
         cmap='Set3', edgecolor='black', linewidth=0.5, legend=True)
axes[1].set_title(f'Max-p Regionalization (p={max_p}, Shape Index={minPartitionShape:.4f})')
axes[1].axis('off')

plt.tight_layout()
plt.show()

## 10. Summary Statistics

In [ ]:
# Print summary for each region
print("Region Summary:")
print("-" * 60)
print(f"{'Region':<10} {'Units':<10} {'Threshold Sum':<15} {'Shape Index':<15}")
print("-" * 60)

for i, region in enumerate(bestPartition.data):
    print(f"{i+1:<10} {len(region.data):<10} {region.spatialAttrTotal:<15.2f} {region.shapeindex:<15.4f}")

print("-" * 60)
print(f"{'Total':<10} {n_polygons:<10} {'':<15} {minPartitionShape:<15.4f}")

## 11. Export Results

In [ ]:
# Save results to files
output_dir = '../results'
os.makedirs(output_dir, exist_ok=True)

# Save as shapefile
gdf.to_file(f'{output_dir}/maxp_result.shp')
print(f"Saved shapefile to {output_dir}/maxp_result.shp")

# Save summary as CSV
summary = pd.DataFrame({
    'max_p': [max_p],
    'shape_index': [minPartitionShape],
    'total_heterogeneity': [bestPartition.totalwithinRegionHetero],
    'threshold': [threshold]
})
summary.to_csv(f'{output_dir}/summary.csv', index=False)
print(f"Saved summary to {output_dir}/summary.csv")

## Done!

The max-p regionalization with compactness constraints has been completed. You can adjust the parameters in Section 4 to experiment with different settings.